In [ ]:
from langchain.chat_models import ChatOpenAI

from langchain.memory import ConversationBufferMemory #대화 기록이 하나씩 쌓이는 메모리, 중복의 경우에도 계속 쌓이기 때문에 비효율적
from langchain.memory import ConversationBufferWindowMemory #대화 기록이 쌓이는 메모리지만, 특정 부분(최근 부분)만 쌓임
from langchain.memory import ConversationSummaryMemory #메시지를 그대로 저장하는 것이 아니라 요약해서 저장(LLM 활용)
from langchain.memory import ConversationSummaryBufferMemory # !!!) 메모리에 최근 내용은 그대로 저장하나, 오래 지난 내용은 요약해서 저장함
from langchain.memory import ConversationKGMemory # Entity추출(필요한 정보는 추출하여 저장)

from langchain.chains import LLMChain

from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

from langchain.schema.runnable import RunnablePassthrough

memory = ConversationBufferMemory()

llm = ChatOpenAI(
    temperature=0.1,
)

memory2 = ConversationBufferWindowMemory(
    return_messages=True,
    k=4    
)

memory3 = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=100,
    return_messages=True, # 실제 메시지를 바꿔주는 속성
    # memory_key="chat_history" 프롬프트에 메모리 입력하는 속성
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


# add_message("Hi I'm Kevin, I'm from Kevin", "Wow that is so cool!")




{'history': "Human: Hi I'm Kevin, I'm from Kevin\nAI: Wow that is so cool!"}

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI taking to a human"),
    MessagesPlaceholder(variable_name="history"), # 메모리 클래스로 변환
    ("human", "{question}"),
])

template = """
    You are a helpful AI talking to a human.

    {history}
    Human:{question}
    AI:
"""

chain = LLMChain(
    llm=llm,
    memory=memory3,
    prompt=prompt,
    verbose=True

)

def load_memory():
    return memory.load_memory_variables({})["history"]

chain2 = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):

    result = chain2.invoke({"question": question})

    memory.save_context({"input": question}, {"output": result.content})

    print(result)


memory.load_memory_variables({})
